## Getting boxscore using nba_api

Goal:
Collect the box scores of every NBA regular game matchup in the 2022-23 season.
Eventually, this will serve to help illustrate the ebs and flows of shooting accuaracy throughout the course of an NBA season.
Could we see whether there are such moments where the league, as a whole, exhibit shooting slumps?

In [ ]:
from nba_api.live.nba.endpoints import boxscore
import pandas as pd
import time

### Function: get_boxscore_dates

In [ ]:
# function returns list of dates of games played in 2022-23 season   
def get_boxscore_dates():
    for i in range(1, 1231):
        gameID = str(i).zfill(5)
        gameIDs = boxscore.BoxScore(gameID_base.format(gameID))
        dates_list.append(gameIDs.get_dict()['game']['gameEt'])
        time.sleep(1.5)

### Forming game ID base

In [ ]:
# Define gameID base value for 2022-23 regular season NBA games 
gameID_base = '00222{}'

# Test boxscore variable
test_box = boxscore.BoxScore('0022200001')

# create a blank dictionary, adopting the box score statistic's keys 
stats_key_dictionary = dict.fromkeys(test_box.get_dict()['game']['homeTeam']['statistics'].keys(), [])

# create blank dataframe
box_df = pd.DataFrame.from_dict(stats_key_dictionary, orient='index').T

## Getting boxscore data

### Loop through each match, collect home and away box score data, and append dictionaries into a single dataframe

Calls for the first half of games failed at game #0022200400

In [ ]:
# loop through each matchup and collect home and away box score data dictionaries
# appends dictionaries into dataframe
# test loop (first half of 2022-23 season games)
for i in range(1,616):
    gameID_last_5_digits = str(i).zfill(5)
    gameID = gameID_base.format(gameID_last_5_digits)
    
    # calls on API to retrieve the boxscores data of the first five games
    box_first_five = boxscore.BoxScore(gameID)
   
    stats_key_dictionary.update(box_first_500.get_dict()['game']['awayTeam']['statistics'])
    box_df = box_df.append(stats_key_dictionary, ignore_index=True)
    print('Home: ', gameID)
    
    stats_key_dictionary.update(box_first_500.get_dict()['game']['homeTeam']['statistics'])
    box_df = box_df.append(stats_key_dictionary, ignore_index=True)
    print('Away: ',gameID)


Error thrown:
```python
C:AppData\Local\Temp\ipykernel_23008\2029473471.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
    box_df = box_df.append(stats_key_dictionary, ignore_index=True)
Home: 0022200399
Away: 0022200399

Traceback (most recent call last):

  File ~\anaconda3\envs\env_3_10_8\lib\site-packages\urllib3\connection.py:174 in _new_conn
    conn = connection.create_connection(

  File ~\anaconda3\envs\env_3_10_8\lib\site-packages\urllib3\util\connection.py:95 in create_connection
    raise err

  File ~\anaconda3\envs\env_3_10_8\lib\site-packages\urllib3\util\connection.py:85 in create_connection
    sock.connect(sa)

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond
```

We'll instead break up the calls into four intervals of 320 games

In [1]:
# loop through each matchup, collect home and away box score data, and append dictionaries to dataframe

#for i in range(1,321):
#for i in range(321,642):
#for i in range(642,963):
#for i in range(963,1231):
    gameID_last_5_digits = str(i).zfill(5)
    gameID = gameID_base.format(gameID_last_5_digits)
    print(gameID)
    
    # calls on API to retrieve the boxscores data of the first five games
    box_call = boxscore.BoxScore(gameID)
   
    stats_key_dictionary.update(box_call.get_dict()['game']['awayTeam']['statistics'])
    box_df = box_df.append(stats_key_dictionary, ignore_index=True)
    
    stats_key_dictionary.update(box_call.get_dict()['game']['homeTeam']['statistics'])
    box_df = box_df.append(stats_key_dictionary, ignore_index=True)
    
    time.sleep(1) #necessary to help mitigate timeouts

IndentationError: unexpected indent (1985646965.py, line 10)

### Getting boxscore ID

In [ ]:
# inserting the game IDs into dataframe
gameID_list = []
for i in range(1, 1231):
    gameID_last5 = str(i).zfill(5)
    gameID = gameID_base.format(gameID_last5)
    gameID_list.append(gameID)
    gameID_list.append(gameID)
    
box_df['Game ID'] = gameID_list

### Creating & Filling Home/Away Data 

In [ ]:
# inserting 'Home/Away' data
box_df['Home/Away'] = box_df.reset_index().apply(lambda x: 'Away' if x['index'] % 2 == 0 else 'Home', axis=1)

### Call on get dates method and duplicate each item in list to match size of dataframe

In [ ]:
dates_list = []
get_boxscore_dates()
dates = [item for item in dates_list for i in range(2)]
box_df.insert(0, 'Date', dates)

## Archiving the Data

In [ ]:
#archive the data
# Save the dataframe to a file or a database of your choice (you can use any format you want)
box_df.to_csv("Spreadsheets/box_score_all.csv", index=False)

## Plotting data

In [ ]:
df = pd.read_csv('Spreadsheets/box_score_all.csv')

## Great alternative, but was getting timeout errors

In [ ]:
# Import the nba_api and pandas modules
import nba_api
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2

# Define the season
season = "2022-23"

# Get all the regular season games for the season
games = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable="Regular Season").get_data_frames()[0]

# Create an empty dataframe to store the box score data
box_score_df = pd.DataFrame()

# Loop through the games and get the box score for each game
for game in games.itertuples():
    # Get the game id
    game_id = game.GAME_ID
    # Get the box score
    box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]
    # Append the box score data to the dataframe
    box_score_df = pd.concat([box_score_df, box_score], ignore_index=True)
    print(game_id)